In [3]:
import pandas as pd
import numpy as np
from IPython.display import display, clear_output, HTML, Image, IFrame
import io
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import requests
from io import BytesIO
from PIL import Image
import urllib.request
import requests as req



In [4]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
my_style = {'description_width': 'initial'}
my_style_two = {'description_width': '150px'}
my_layout = {'width': '600px'}
my_layout_short = {'width': '300px'}
my_layout_shortest = {'width': '200px'}
FLAG__ON_BINDER = True
FLAG__FROM_GIT = True

DATA_FRAME_PATH_LOCAL = './data_to_display.csv'
DATA_FRAME_PATH_GIT_URL = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/'
DATA_FRAME_PATH_LOCAL_BINDER  = '/home/jovyan/cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'
    

In [6]:
def image_to_byte_array(image:Image):
  imgByteArr = io.BytesIO()
  image.save(imgByteArr, format=image.format)
  imgByteArr = imgByteArr.getvalue()
  return imgByteArr

In [7]:
data_to_display_df = pd.read_csv(DATA_FRAME_PATH_LOCAL, index_col = False)
data_to_display_df = data_to_display_df.drop("Unnamed: 0", axis = 1).drop("id", axis = 1)

data_to_display_df['title_lower'] = data_to_display_df["title"].apply(lambda string: string.lower())
# display(data_to_display_df.head())

# print("Num rows", data_to_display_df.shape[0])
# print("Num cols",data_to_display_df.shape[1])

In [8]:
file_size_list = data_to_display_df.size_kb.unique()
max_usability_rating = data_to_display_df.usabilityRating.max()
min_usability_rating = data_to_display_df.usabilityRating.min()

my_intro_html= """<h4>Using the audit tool</h4><p>This tool will randomly select and display a sentence which
has already been classified by the model thus facilitating human evaluation and correction of model predictions.
These validated and corrected predictions will be fed back to the model as part of a re-training process, with 
the objective of increasing model accuracy. The tool will automatically select and display a random sentence from
the pool each time an action is involked, either by modifying the filter or by one of the three action buttons
[Change], [Ok] and [Skip]. Once the sentence is audited via either the [Ok] or [Change] button it will be used to
train the model on the next training iteration.<br/>[Skip]: Use the Skip button if you are unable to confirm
    the classification (no action will be taken)</p><p>[Ok]: </span> Use the OK button to confirm that the 
        sentence has been classified correctly as [Positive] or [Negative] for the convention in question i.e 
        if the classification is True </p><p>[Chang]e]: Use the change button, if you believe the sentence has
            been classified incorrectly by the model  i.e if the classification is a "False Positive" or 
            "False Negative".<br/> If the original classification has been "False Positive" and you can correctly
            classify the sentence as belonging to another convention, please select the appropriate convention 
            via the [Change to] dropdown.</p><p><em>Note: A&nbsp;<strong>GitHub</strong>&nbsp;username and 
                password is required to save the results of the audits, it is important to remember to save the
                audit on a regular basis</em></p><hr/><p></p> """                             


#image
IMAGE_PATH = 'http://www.euro.who.int/__data/assets/image/0020/392042/SDG3-600-collage.png'
image = Image.open(urllib.request.urlopen(IMAGE_PATH))
image = image_to_byte_array(image)


In [9]:
selectHTMLHead = widgets.HTML(
            "<font color = '#8c8c8c'><h3><center>*** Model Classification, Audit Tool ***</center></h3></font>")

selectHTML = widgets.HTML(
            "<font color = '#8c8c8c'><h4><left>--- Filter ---</left></h4></font>")

space = widgets.Label('  ', layout=widgets.Layout(width='100%'))




accordion = widgets.Accordion(children=
                              [widgets.HTML(my_intro_html),
                               widgets.Image(value=image,format='png',width=1200,height=1200),
                              ])


accordion.set_title(0, 'My title stuff')
accordion.set_title(1, 'My image whatever it is')
accordion.selected_index = 0

file_size_list = data_to_display_df.size_kb.unique()
file_size_list = np.insert(file_size_list, 0, "Any")
file_size_drop_down = widgets.Dropdown(
    options =  file_size_list,
    description="File size :",
     #value=None,
    disabled=False,
    style = my_style_two,
    layout = my_layout
)



usability_rating_slider_min = widgets.FloatSlider(
    min=min_usability_rating, 
    max=max_usability_rating,
    value = min_usability_rating,
    step = 0.01,
    description='Min Usability level:',
    style = my_style_two,
    layout = my_layout
    )
usability_rating_slider_min.style.handle_color = '#5c85d6'


usability_rating_slider_max = widgets.FloatSlider(
    min=min_usability_rating, 
    max=max_usability_rating, 
    value = max_usability_rating,
    step = 0.01,
    description='Max Usability level:',
    style = my_style_two,
    layout = my_layout)
usability_rating_slider_max.style.handle_color = '#5c85d6'



search_box_title = widgets.Text(value='', 
                              description='Search Title:',
                              style = my_style_two,
                              layout = my_layout)


forward_button = widgets.Button(
            description= "Forward",
            disabled=False,
            button_style='',
            tooltip='View next dataset')
forward_button.style.button_color = '#e68a00'



back_button = widgets.Button(
            description='Back',
            disabled=False,
            button_style='',
             tooltip='View previous dataset')
back_button.style.button_color = '#d9d9d9'

last_clicked = "forward"

def forward_button_clicked(b):
    global data_to_display_df
    global last_clicked
    last_clicked = "forward"
   
    with output:
        clear_output(wait=True)
        print ("Forward button clicked") 
        output_function(file_size_drop_down.value,
                        usability_rating_slider_min.value,
                        usability_rating_slider_max.value,
                        search_box_title.value)
        

def back_button_clicked(b):
    global data_to_display_df
    global last_clicked
    last_clicked = "back"
   
    with output:
        clear_output(wait=True)
        print ("Back button clicked") 
        output_function(file_size_drop_down.value,
                        usability_rating_slider_min.value,
                        usability_rating_slider_max.value,
                        search_box_title.value)
        


            
            
def output_function(file_size_drop_down_value,
                        usability_rating_slider_min_value,
                        usability_rating_slider_max_value,
                        search_box_title_value):
    
#     print ("output function called")
    global data_to_display_df
    global last_clicked
    file_size_drop_down.disable = False
    usability_rating_slider_min.disable = False
    usability_rating_slider_max.disable = False
    search_box_title.disable = False
    
#     display (data_to_display_df.head())
    
    print (file_size_drop_down_value)
    print (usability_rating_slider_min_value)
    print (usability_rating_slider_max_value)
    print (search_box_title_value)
    print (last_clicked)
    
    row_to_display = pd.DataFrame()
    
    if search_box_title_value != '':
        file_size_drop_down.disable = True
        usability_rating_slider_min.disable = True
        usability_rating_slider_max.disable = True
        filtered_result =  data_to_display_df.loc[(data_to_display_df['title_lower'].str.contains(search_box_title_value))]
                                              
    
    elif file_size_drop_down_value == "Any":
        filtered_result =  data_to_display_df.loc[(data_to_display_df['usabilityRating'] > usability_rating_slider_min_value ) &
                                              (data_to_display_df['usabilityRating'] < usability_rating_slider_max_value )]
     
    
    
    else:
        filtered_result =  data_to_display_df.loc[(data_to_display_df['size_kb'] == file_size_drop_down_value) &
                                              (data_to_display_df['usabilityRating'] > usability_rating_slider_min_value ) &
                                              (data_to_display_df['usabilityRating'] < usability_rating_slider_max_value )]
         
#     display ( filtered_result.head())
    print ( "Num filtered rows", filtered_result.shape[0])
    
    if not filtered_result.empty:
        
#         row_to_display = filtered_result.iloc[current_position]
        row_to_display = filtered_result.sample(n = 1)
        
        title = row_to_display.iloc[0].title
        subtitle = row_to_display.iloc[0].subtitle
        rating = row_to_display.iloc[0].usabilityRating
        size = row_to_display.iloc[0].size_kb
    
        kaggle_url = row_to_display.iloc[0].url
        zip_file_url = "https://www." + kaggle_url + "/download"
        url = "https://www." + kaggle_url

    
        txt = "The number of filtered samples is " + str(filtered_result.shape[0])
        display(HTML("<font color='purple'>" +  txt + "</font>"))
        display(HTML("<font color='green'>" + "Title: " + title + "</font>"))
        display(HTML("<font color='green'>" + "Sub Title: " + subtitle + "</font>"))
        display(HTML("<font color='green'>" + "Usability Rating: " + str(round(rating, 2)) + "</font>"))
        display(HTML("<font color='green'>" + "File Size: " + size + "</font>"))
        print ("Kaggle URL", url)
        print ("Download Data:",  zip_file_url)
    else:
        print ("Relax your filter baby")
       
        
                       
    
output = widgets.interactive_output(output_function, {'file_size_drop_down_value' :file_size_drop_down,
                                    'usability_rating_slider_min_value' :usability_rating_slider_min,
                                    'usability_rating_slider_max_value':usability_rating_slider_max,
                                    'search_box_title_value':search_box_title})

forward_button.on_click(forward_button_clicked)
back_button.on_click(back_button_clicked)


display(
        selectHTMLHead,
        accordion,
        space,
        selectHTML,
        space,
        widgets.VBox([file_size_drop_down,
                        usability_rating_slider_min,
                        usability_rating_slider_max,
                        search_box_title]),
        
        space,
        widgets.HBox([back_button,forward_button]),
        output
        
    )

HTML(value="<font color = '#8c8c8c'><h3><center>*** Model Classification, Audit Tool ***</center></h3></font>"…

Accordion(children=(HTML(value='<h4>Using the audit tool</h4><p>This tool will randomly select and display a s…

Label(value='  ', layout=Layout(width='100%'))

HTML(value="<font color = '#8c8c8c'><h4><left>--- Filter ---</left></h4></font>")

Label(value='  ', layout=Layout(width='100%'))

Label(value='  ', layout=Layout(width='100%'))

Output()